In [1]:
import pandas as pd
import folium
from pyproj import Transformer

In [2]:
stations = pd.read_csv('netatmo_stations.csv', delimiter=',')
stations.head()

,device_id,module_id,lon,lat
0,70:ee:50:af:48:36,02:00:00:af:97:b6,-5.821514,43.380573
1,70:ee:50:12:ce:a6,02:00:00:12:ba:e6,-5.821986,43.381939
2,70:ee:50:a2:20:c2,02:00:00:a2:03:0c,-5.816930,43.378882
3,70:ee:50:71:49:30,02:00:00:71:4b:f2,-5.811817,43.377968
4,70:ee:50:90:d8:82,02:00:00:ab:de:78,-5.804601,43.379862


In [3]:
zgz_coords = (41.653858, -0.876258)
map = folium.Map(location=zgz_coords, zoom_start=13)

# Iterate over each station
for index, station in stations.iterrows():
    lon = station['lon']
    lat = station['lat']

    # Add a marker for the point
    folium.Marker([lat, lon], popup=station['device_id']).add_to(map)

# If running in a Jupyter Notebook, display the map inline
map

In [4]:
'''ref_stations = [
    '70:ee:50:2b:5b:da', # Villamar
    '70:ee:50:5e:f5:44', # Faro de Baxo
    '70:ee:50:3a:22:2c'  # Las campas
]'''

"ref_stations = [\n    '70:ee:50:2b:5b:da', # Villamar\n    '70:ee:50:5e:f5:44'\n]"

### Generate shapefile

In [5]:
import geopandas as gpd
from shapely.geometry import Point

In [6]:
# Create a GeoDataFrame
geometry = [Point(xy) for xy in zip(stations['lon'], stations['lat'])]
gdf = gpd.GeoDataFrame(stations, geometry=geometry)

# Set CRS and save
filename = 'netatmo_stations.shp'
gdf.set_crs(epsg=4326, inplace=True)
gdf = gdf.to_crs('EPSG:25830')
gdf.to_file(filename)

### Locate stations

In [10]:
station_id = '70:ee:50:2b:5b:da'

row = stations[stations['device_id'] == station_id]

# Extract lon and lat
lon, lat = row[['lon', 'lat']].values[0]

location_map = folium.Map(location=zgz_coords, zoom_start=13)
folium.Marker([lat, lon], popup=station_id).add_to(location_map)
location_map